<a href="https://colab.research.google.com/github/pranjalchaubey/Predicting-Stock-Direction/blob/master/src/Stock_Direction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Direction
In this notebook we are going to use Random Forests to predict the direction  
of stocks for the next trading day.  
In more technical terms, we are going to try and predict the _**Momentum**_  
of the stocks.  
Based on the momentum, we are going to give the following signals to the user  
1. Buy (_+ve momentum_)
2. Sell (_-ve momentum_)
3. Hold (_neutral_)  

This system will work on the daily closing prices and will generate the signals by predicting the momentum for the next trading day. 

##Setup The Colab Environment 
Install and Import the required libraries.  

In [0]:
# Debug Flag
GLOBAL_DEBUG = True

In [2]:
# Install the yFinance library 
!pip install yfinance 

  Created wheel for yfinance: filename=yfinance-0.1.45-cp36-none-any.whl size=14652 sha256=aaab43d8d2ccd1820638093cb912931161867cf417199759707302b6fa44ff9b
  Stored in directory: /root/.cache/pip/wheels/0c/d1/df/aa9a7744a4ac353cc9a1f2c3aaea7c1f457fc49de4286f2d88
Successfully built yfinance


In [0]:
# yFinance will help us fetch the data for our dataset
import yfinance as yf

In [0]:
# Data Mining and plotting 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
# Classifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os
import sys
import shutil
# if in Google Colaboratory
try:
    from google.colab import drive
except:
    pass

import warnings
warnings.filterwarnings("ignore")

In [0]:
# Configure Pandas Display Options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [8]:
# Since I am running this notebook on Colab, 
# Let's try and get some system information 
import platform
print('System Processor: ', platform.processor(), '\n')
!nvidia-smi

System Processor:  x86_64 

Sun Sep 29 20:31:13 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                

## Load the Data
We will extract the 5 year history of 10 stocks chosen by the user. 

<br/>Here, we will take the following stocks as an example
1. Facebook FB
2. Apple AAPL
3. Amazon AMZN
4. Netflix NFLX
5. Google GOOGL
6. Starbucks SBUX
7. Exxon Mobil XOM
8. Johnson & Johnson JNJ
9. Bank of America BAC
10. General Motors GM

<br/>In the actual application, stock selection will be done by the user. 

In [0]:
# List of stock tickers 
# this info will come from the user, perhaps in the form of a pickle file 
ticker_list = ['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'SBUX', 'XOM', 'JNJ',\
               'BAC', 'GM']

In [0]:
def fetch_ticker_data(tickers): 
  """
  Fetch 5 years of historical data for the 'tickers', from Yahoo Finance.    

  Parameters
  ----------
  tickers : List   
    List of stocks chosen by the user.   
      
  Returns
  -------
  data : Dataframe 
    Pandas df with historical stock data time series. 
  """
  # Debug 
  LOCAL_DEBUG = True
  
  # Get the Data from Yahoo! Finance
  data = yf.download(tickers, start="2017-01-01", end="2017-04-30")
  if GLOBAL_DEBUG and LOCAL_DEBUG:
    display(data.head(20))
  
  # Data is multi-indexed on the columns 
  # Make it multi-index on the rows, to make it 
  # fit for consumption by the RandomForestClassiffier
  data = data.stack(1) 
  if GLOBAL_DEBUG and LOCAL_DEBUG:
    display(data.head(20))

  # Drop all the columns except for Adj Close & Volume
  data = data[['Adj Close', 'Volume']]
  # Rename the column names 
  data.columns = ['Close', 'Volume']  
  if GLOBAL_DEBUG and LOCAL_DEBUG:
    display(data.head(20))

  return data

In [27]:
# Go fetch! 
dataset = fetch_ticker_data(tickers=ticker_list)

[*********************100%***********************]  10 of 10 downloaded


Adj Close                                                                       Close                                                                        High                                                                         Low                                                                        Open                                                                        Volume                                                                                         
                AAPL    AMZN    BAC      FB     GM   GOOGL     JNJ    NFLX   SBUX    XOM    AAPL    AMZN    BAC      FB     GM   GOOGL     JNJ    NFLX   SBUX    XOM    AAPL    AMZN    BAC      FB     GM   GOOGL     JNJ    NFLX   SBUX    XOM    AAPL    AMZN    BAC      FB     GM   GOOGL     JNJ    NFLX   SBUX    XOM    AAPL    AMZN    BAC      FB     GM   GOOGL     JNJ    NFLX   SBUX    XOM      AAPL     AMZN        BAC        FB        GM    GOOGL       JNJ      NFLX      SBUX       XOM
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
2017-01-03    111.29  753.67  21.41  116.86  31.13  808.01  107.65  127.49  52.39  81.20  116.15  753.67  22.53  116.86  35.15  808.01  115.84  127.49  55.35  90.89  116.33  758.76  22.68  117.84  35.57  811.44  116.00  128.19  55.95  91.34  114.76  747.70  22.20  115.51  34.84  796.89  115.23  124.31  55.04  90.15  115.80  757.92  22.60  116.03  34.98  800.62  115.78  124.96  55.91  90.94  28781900  3521100   99298100  20663900  10904900  1959000   5953000   9437900   7809300  10360600
2017-01-04    111.16  757.18  21.81  118.69  32.85  807.77  107.48  129.41  53.00  80.30  116.02  757.18  22.95  118.69  37.09  807.77  115.65  129.41  55.99  89.89  116.51  759.68  22.96  119.66  37.24  813.43  116.23  130.17  56.20  91.15  115.75  754.20  22.60  117.29  35.47  804.11  115.31  126.55  55.38  89.75  115.85  758.39  22.72  117.55  35.60  809.89  115.50  127.49  55.56  91.12  21118100  2510500   76875100  19630900  23388500  1515300   5828900   7843600   7796300   9434200
2017-01-05    111.73  780.45  21.55  120.67  32.23  813.02  108.60  131.81  53.44  79.11  116.61  780.45  22.68  120.67  36.39  813.02  116.86  131.81  56.46  88.55  116.86  782.40  22.93  120.95  37.05  813.74  116.97  132.75  56.53  90.30  115.81  760.26  22.35  118.32  36.07  805.92  115.44  128.90  55.81  88.44  115.92  761.55  22.82  118.86  37.01  807.50  116.00  129.22  56.08  90.19  22193600  5830100   86826400  19492200  15636700  1340500   6217200  10185500   7602300  14443200
2017-01-06    112.97  795.99  21.55  123.41  31.88  825.21  108.08  131.07  54.08  79.06  117.91  795.99  22.68  123.41  35.99  825.21  116.30  131.07  57.13  88.50  118.16  799.44  22.85  123.88  36.55  828.96  117.00  133.88  57.27  88.98  116.47  778.48  22.56  120.03  35.93  811.50  115.76  129.81  56.08  87.82  116.78  782.36  22.78  120.98  36.41  814.99  116.67  132.08  56.63  88.79  31751900  5986200   66281500  28545300  13240100  2017100   5221400  10657900   8587800  16518100
2017-01-09    114.01  796.92  21.43  124.90  31.90  827.18  108.06  130.95  55.09  77.76  118.99  796.92  22.55  124.90  36.01  827.18  116.28  130.95  58.20  87.04  119.43  801.77  22.71  125.43  36.53  830.43  116.61  131.99  58.34  88.22  117.94  791.77  22.40  123.04  35.86  821.62  115.89  129.89  57.25  86.55  117.95  798.00  22.51  123.55  36.12  826.37  116.35  131.48  57.26  88.22  33561900  3446100   75901500  22880400  15204500  1408900   5457500   5771800  12640500  13762300
2017-01-10    114.12  795.90  21.80  124.35  33.08  826.01  107.95  129.89 

Adj Close   Close    High     Low    Open    Volume
Date                                                                 
2017-01-03 AAPL      111.29  116.15  116.33  114.76  115.80  28781900
           AMZN      753.67  753.67  758.76  747.70  757.92   3521100
           BAC        21.41   22.53   22.68   22.20   22.60  99298100
           FB        116.86  116.86  117.84  115.51  116.03  20663900
           GM         31.13   35.15   35.57   34.84   34.98  10904900
           GOOGL     808.01  808.01  811.44  796.89  800.62   1959000
           JNJ       107.65  115.84  116.00  115.23  115.78   5953000
           NFLX      127.49  127.49  128.19  124.31  124.96   9437900
           SBUX       52.39   55.35   55.95   55.04   55.91   7809300
           XOM        81.20   90.89   91.34   90.15   90.94  10360600
2017-01-04 AAPL      111.16  116.02  116.51  115.75  115.85  21118100
           AMZN      757.18  757.18  759.68  754.20  758.39   2510500
           BAC        21.81   22.95   22.96   22.60   22.72  76875100
           FB        118.69  118.69  119.66  117.29  117.55  19630900
           GM         32.85   37.09   37.24   35.47   35.60  23388500
           GOOGL     807.77  807.77  813.43  804.11  809.89   1515300
           JNJ       107.48  115.65  116.23  115.31  115.50   5828900
           NFLX      129.41  129.41  130.17  126.55  127.49   7843600
           SBUX       53.00   55.99   56.20   55.38   55.56   7796300
           XOM        80.30   89.89   91.15   89.75   91.12   9434200

Close    Volume
Date                              
2017-01-03 AAPL   111.29  28781900
           AMZN   753.67   3521100
           BAC     21.41  99298100
           FB     116.86  20663900
           GM      31.13  10904900
           GOOGL  808.01   1959000
           JNJ    107.65   5953000
           NFLX   127.49   9437900
           SBUX    52.39   7809300
           XOM     81.20  10360600
2017-01-04 AAPL   111.16  21118100
           AMZN   757.18   2510500
           BAC     21.81  76875100
           FB     118.69  19630900
           GM      32.85  23388500
           GOOGL  807.77   1515300
           JNJ    107.48   5828900
           NFLX   129.41   7843600
           SBUX    53.00   7796300
           XOM     80.30   9434200